In [1]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
import pandas as pd  # noqa: E402

In [2]:
def get_results(dataset, disc_model, method, columns):
    print(dataset, disc_model, method)

    df = pd.DataFrame(columns=columns)

    for fold_n in range(5):
        try:
            metrics_path = f"../models/{dataset}/{method}/fold_{fold_n}/cf_metrics_{disc_model}.csv"
            df_ = pd.read_csv(metrics_path)
            df = pd.concat([df, df_], axis=0)
        except Exception as e:
            pass
            # print(f"File not found: {metrics_path}")
            # df_ = pd.DataFrame(columns=columns)
            # df = pd.concat([df, df_], axis=0)
    # print(df.shape)
    df["dataset"] = dataset
    df["disc_model"] = disc_model
    df["method"] = method

    return df

In [3]:
columns = [
    "dataset",
    "disc_model",
    "method",
    "K_vectors",
    "validity",
    "prob_plausibility",
    "cf_belongs_to_group",
    "log_density_cf",
    "proximity_continuous_manhattan",
    "proximity_continuous_euclidean",
    "isolation_forest_scores_cf",
    "lof_scores_cf",
    "time",
]
datasets = [
    "BlobsDataset",
    "LawDataset",
    "MoonsDataset",
    "WineDataset",
    "HelocDataset",
    "DigitsDataset",
]
disc_models = ["MultinomialLogisticRegression", "MultilayerPerceptron"]
global_methods = ["AReS", "GLOBE_CE", "RPPCEF_GLOBAL"]  # "GCE"
local_methods = ["wach", "Artelt", "RPPCEF_LOCAL"]  # "PPCEF_2"
group_methods = ["RPPCEF"]  # "GLANCE",
methods = global_methods + local_methods + group_methods

dataset = datasets[5]
disc_model = disc_models[0]
# method = local_methods[2]

df_all = pd.DataFrame(columns=columns)
for disc_model in disc_models:
    for dataset in datasets:
        for method in global_methods:
            df = get_results(dataset, disc_model, method, columns)
            df_all = pd.concat([df_all, df], axis=0)

BlobsDataset MultinomialLogisticRegression AReS
BlobsDataset MultinomialLogisticRegression GLOBE_CE
BlobsDataset MultinomialLogisticRegression RPPCEF_GLOBAL
LawDataset MultinomialLogisticRegression AReS
LawDataset MultinomialLogisticRegression GLOBE_CE
LawDataset MultinomialLogisticRegression RPPCEF_GLOBAL
MoonsDataset MultinomialLogisticRegression AReS
MoonsDataset MultinomialLogisticRegression GLOBE_CE
MoonsDataset MultinomialLogisticRegression RPPCEF_GLOBAL
WineDataset MultinomialLogisticRegression AReS
WineDataset MultinomialLogisticRegression GLOBE_CE
WineDataset MultinomialLogisticRegression RPPCEF_GLOBAL
HelocDataset MultinomialLogisticRegression AReS
HelocDataset MultinomialLogisticRegression GLOBE_CE
HelocDataset MultinomialLogisticRegression RPPCEF_GLOBAL
DigitsDataset MultinomialLogisticRegression AReS
DigitsDataset MultinomialLogisticRegression GLOBE_CE
DigitsDataset MultinomialLogisticRegression RPPCEF_GLOBAL
BlobsDataset MultilayerPerceptron AReS
BlobsDataset MultilayerPe

In [5]:
# df_all.to_csv("results_GLOBAL.csv", index=False)

In [4]:
df_all.groupby(["disc_model", "dataset", "method"]).mean().round(2)

K_vectors  validity  \
disc_model                    dataset       method                              
MultilayerPerceptron          BlobsDataset  GLOBE_CE            NaN      0.99   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              DigitsDataset GLOBE_CE            NaN      0.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              HelocDataset  AReS                NaN      0.28   
                                            GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              LawDataset    GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              MoonsDataset  GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      0.91   
                              WineDataset   GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
MultinomialLogisticRegression BlobsDataset  GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              DigitsDataset GLOBE_CE            NaN      0.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              HelocDataset  AReS                NaN      0.18   
                                            GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              LawDataset    GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              MoonsDataset  GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      1.00   
                              WineDataset   GLOBE_CE            NaN      1.00   
                                            RPPCEF_GLOBAL       1.0      1.00   

                                                           prob_plausibility  \
disc_model                    dataset       method                             
MultilayerPerceptron          BlobsDataset  GLOBE_CE                    0.00   
                                            RPPCEF_GLOBAL               0.92   
                              DigitsDataset GLOBE_CE                     NaN   
                                            RPPCEF_GLOBAL               0.72   
                              HelocDataset  AReS                        0.18   
                                            GLOBE_CE                    0.17   
                                            RPPCEF_GLOBAL               0.46   
                              LawDataset    GLOBE_CE                    0.37   
                                            RPPCEF_GLOBAL               0.79   
                              MoonsDataset  GLOBE_CE                    0.00   
                                            RPPCEF_GLOBAL               0.63   
                              WineDataset   GLOBE_CE                    0.00   
                                            RPPCEF_GLOBAL               0.95   
MultinomialLogisticRegression BlobsDataset  GLOBE_CE                    0.00   
                                            RPPCEF_GLOBAL               0.92   
                              DigitsDataset GLOBE_CE                    0.00   
                                            RPPCEF_GLOBAL               0.69   
                              HelocDataset  AReS                        0.07   
                                            GLOBE_CE                    0.13   
                                            RPPCEF_GLOBAL               0.46   
                             

In [6]:
columns = [
    "dataset",
    "disc_model",
    "method",
    "K_vectors",
    "validity",
    "prob_plausibility",
    "cf_belongs_to_group",
    "log_density_cf",
    "proximity_continuous_manhattan",
    "proximity_continuous_euclidean",
    "isolation_forest_scores_cf",
    "lof_scores_cf",
    "time",
]
datasets = [
    "BlobsDataset",
    "LawDataset",
    "MoonsDataset",
    "WineDataset",
    "HelocDataset",
    "DigitsDataset",
]
disc_models = ["MultinomialLogisticRegression", "MultilayerPerceptron"]
global_methods = ["AReS", "GLOBE_CE", "RPPCEF_GLOBAL"]  # "GCE"
local_methods = ["wach", "Artelt", "RPPCEF_LOCAL"]  # "PPCEF_2"
group_methods = ["RPPCEF"]  # "GLANCE",
methods = global_methods + local_methods + group_methods

dataset = datasets[5]
disc_model = disc_models[0]
# method = local_methods[2]

df_all = pd.DataFrame(columns=columns)
for disc_model in disc_models:
    for dataset in datasets:
        for method in local_methods:
            df = get_results(dataset, disc_model, method, columns)
            df_all = pd.concat([df_all, df], axis=0)

BlobsDataset MultinomialLogisticRegression wach
BlobsDataset MultinomialLogisticRegression Artelt
BlobsDataset MultinomialLogisticRegression RPPCEF_LOCAL
LawDataset MultinomialLogisticRegression wach
LawDataset MultinomialLogisticRegression Artelt
LawDataset MultinomialLogisticRegression RPPCEF_LOCAL
MoonsDataset MultinomialLogisticRegression wach
MoonsDataset MultinomialLogisticRegression Artelt
MoonsDataset MultinomialLogisticRegression RPPCEF_LOCAL
WineDataset MultinomialLogisticRegression wach
WineDataset MultinomialLogisticRegression Artelt
WineDataset MultinomialLogisticRegression RPPCEF_LOCAL
HelocDataset MultinomialLogisticRegression wach
HelocDataset MultinomialLogisticRegression Artelt
HelocDataset MultinomialLogisticRegression RPPCEF_LOCAL
DigitsDataset MultinomialLogisticRegression wach
DigitsDataset MultinomialLogisticRegression Artelt
DigitsDataset MultinomialLogisticRegression RPPCEF_LOCAL
BlobsDataset MultilayerPerceptron wach
BlobsDataset MultilayerPerceptron Artelt
Bl

In [8]:
# df_all.to_csv("results_LOCAL.csv", index=False)

In [7]:
df_all.groupby(["disc_model", "dataset", "method"]).mean().round(2)

K_vectors  validity  \
disc_model                    dataset       method                             
MultilayerPerceptron          BlobsDataset  RPPCEF_LOCAL     100.2      1.00   
                                            wach               NaN      0.77   
                              DigitsDataset RPPCEF_LOCAL     110.4      1.00   
                              HelocDataset  RPPCEF_LOCAL    1043.4      1.00   
                                            wach               NaN      1.00   
                              LawDataset    RPPCEF_LOCAL     219.2      1.00   
                                            wach               NaN      1.00   
                              MoonsDataset  RPPCEF_LOCAL     102.4      1.00   
                                            wach               NaN      1.00   
                              WineDataset   RPPCEF_LOCAL      11.6      1.00   
                                            wach               NaN      0.92   
MultinomialLogisticRegression BlobsDataset  Artelt             NaN      1.00   
                                            RPPCEF_LOCAL     100.4      1.00   
                                            wach               NaN      0.90   
                              DigitsDataset Artelt             NaN      1.00   
                                            RPPCEF_LOCAL     110.8      1.00   
                                            wach               NaN      0.00   
                              HelocDataset  RPPCEF_LOCAL    1037.2      1.00   
                                            wach               NaN      1.00   
                              LawDataset    Artelt             NaN      1.00   
                                            RPPCEF_LOCAL     220.8      1.00   
                                            wach               NaN      1.00   
                              MoonsDataset  Artelt             NaN      1.00   
                                            RPPCEF_LOCAL     102.6      1.00   
                                            wach               NaN      1.00   
                              WineDataset   Artelt             NaN      1.00   
                                            RPPCEF_LOCAL      11.6      1.00   
                                            wach               NaN      1.00   

                                                          prob_plausibility  \
disc_model                    dataset       method                            
MultilayerPerceptron          BlobsDataset  RPPCEF_LOCAL               1.00   
                                            wach                       0.00   
                              DigitsDataset RPPCEF_LOCAL               1.00   
                              HelocDataset  RPPCEF_LOCAL               1.00   
                                            wach                       0.60   
                              LawDataset    RPPCEF_LOCAL               1.00   
                                            wach                       0.45   
                              MoonsDataset  RPPCEF_LOCAL               1.00   
                                            wach                       0.00   
                              WineDataset   RPPCEF_LOCAL               1.00   
                                            wach                       0.08   
MultinomialLogisticRegression BlobsDataset  Artelt                     0.00   
                                            RPPCEF_LOCAL               1.00   
                                            wach                       0.00   
                              DigitsDataset Artelt                     0.00   
                                            RPPCEF_LOCAL               1.00   
                                            wach                        NaN   
                              HelocDataset  RPPCEF_LOCAL               1.00   
                                            wach                       0.00 

In [9]:
columns = [
    "dataset",
    "disc_model",
    "method",
    "K_vectors",
    "validity",
    "prob_plausibility",
    "cf_belongs_to_group",
    "log_density_cf",
    "proximity_continuous_manhattan",
    "proximity_continuous_euclidean",
    "isolation_forest_scores_cf",
    "lof_scores_cf",
    "time",
]
datasets = [
    "BlobsDataset",
    "LawDataset",
    "MoonsDataset",
    "WineDataset",
    "HelocDataset",
    "DigitsDataset",
]
disc_models = ["MultinomialLogisticRegression", "MultilayerPerceptron"]
global_methods = ["AReS", "GLOBE_CE", "RPPCEF_GLOBAL"]  # "GCE"
local_methods = ["wach", "Artelt", "RPPCEF_LOCAL"]  # "PPCEF_2"
group_methods = ["RPPCEF"]  # "GLANCE",
methods = global_methods + local_methods + group_methods

dataset = datasets[5]
disc_model = disc_models[0]
# method = local_methods[2]

df_all = pd.DataFrame(columns=columns)
for disc_model in disc_models:
    for dataset in datasets:
        for method in group_methods:
            df = get_results(dataset, disc_model, method, columns)
            df_all = pd.concat([df_all, df], axis=0)

BlobsDataset MultinomialLogisticRegression RPPCEF
LawDataset MultinomialLogisticRegression RPPCEF
MoonsDataset MultinomialLogisticRegression RPPCEF
WineDataset MultinomialLogisticRegression RPPCEF
HelocDataset MultinomialLogisticRegression RPPCEF
DigitsDataset MultinomialLogisticRegression RPPCEF
BlobsDataset MultilayerPerceptron RPPCEF
LawDataset MultilayerPerceptron RPPCEF
MoonsDataset MultilayerPerceptron RPPCEF
WineDataset MultilayerPerceptron RPPCEF
HelocDataset MultilayerPerceptron RPPCEF
DigitsDataset MultilayerPerceptron RPPCEF


In [10]:
df_all.groupby(["disc_model", "dataset", "method"]).mean().round(2)

K_vectors  validity  \
disc_model                    dataset       method                       
MultilayerPerceptron          BlobsDataset  RPPCEF       1.0      1.00   
                              DigitsDataset RPPCEF       4.0      1.00   
                              HelocDataset  RPPCEF      10.0      0.98   
                              LawDataset    RPPCEF       2.0      1.00   
                              MoonsDataset  RPPCEF       2.4      1.00   
                              WineDataset   RPPCEF       1.4      1.00   
MultinomialLogisticRegression BlobsDataset  RPPCEF       1.0      1.00   
                              DigitsDataset RPPCEF       4.0      1.00   
                              HelocDataset  RPPCEF      10.0      1.00   
                              LawDataset    RPPCEF       1.2      1.00   
                              MoonsDataset  RPPCEF       2.0      1.00   
                              WineDataset   RPPCEF       1.2      1.00   

                                                    prob_plausibility  \
disc_model                    dataset       method                      
MultilayerPerceptron          BlobsDataset  RPPCEF               0.92   
                              DigitsDataset RPPCEF               0.83   
                              HelocDataset  RPPCEF               0.18   
                              LawDataset    RPPCEF               0.85   
                              MoonsDataset  RPPCEF               0.92   
                              WineDataset   RPPCEF               1.00   
MultinomialLogisticRegression BlobsDataset  RPPCEF               0.92   
                              DigitsDataset RPPCEF               0.85   
                              HelocDataset  RPPCEF               0.16   
                              LawDataset    RPPCEF               0.83   
                              MoonsDataset  RPPCEF               0.77   
                              WineDataset   RPPCEF               1.00   

                                                    cf_belongs_to_group  \
disc_model                    dataset       method                        
MultilayerPerceptron          BlobsDataset  RPPCEF                 1.00   
                              DigitsDataset RPPCEF                 0.85   
                              HelocDataset  RPPCEF                 0.99   
                              LawDataset    RPPCEF                 0.97   
                              MoonsDataset  RPPCEF                 0.88   
                              WineDataset   RPPCEF                 0.98   
MultinomialLogisticRegression BlobsDataset  RPPCEF                 1.00   
                              DigitsDataset RPPCEF                 0.85   
                              HelocDataset  RPPCEF                 0.99   
                              LawDataset    RPPCEF                 0.99   
                              MoonsDataset  RPPCEF                 0.99   
                              WineDataset   RPPCEF                 1.00   

                                                    log_density_cf  \
disc_model                    dataset       method                   
MultilayerPerceptron          BlobsDataset  RPPCEF            2.88   
                              DigitsDataset RPPCEF          -99.00   
                              HelocDataset  RPPCEF           14.96   
                              LawDataset    RPPCEF            1.70   
                              MoonsDataset  RPPCEF            1.67   
                              WineDataset   RPPCEF            7.86   
MultinomialLogisticRegression BlobsDataset  RPPCEF            2.86   
                              DigitsDataset RPPCEF          -99.04   
                              HelocDataset  RPPCEF           18.52   
                              LawDataset    RPPCEF            1.67   
                              MoonsDataset  RPPCEF            1.24   
                              

In [11]:
df_all.to_csv("results_GROUP.csv", index=False)

In [33]:
columns_map = {
    "dataset": "dataset",
    "disc_model": "model",
    "method": "method",
    "validity": "Validity",
    "proximity_continuous_euclidean": "L2",
    "prob_plausibility": "Prob. Plaus.",
    "log_density_cf": "Log Density",
    "isolation_forest_scores_cf": "IsoForest",
    "lof_scores_cf": "LOF",
    "time": "Time",
}
df_global = pd.read_csv("results_GLOBAL.csv")
df_global = df_global.rename(columns=columns_map)[columns_map.values()]
df_global_mean = df_global.groupby(["model", "dataset", "method"]).mean().round(2)
df_global_std = df_global.groupby(["model", "dataset", "method"]).std().round(2)

for column in df_global_mean.columns:
    df_global_mean[column] = (
        "$"
        + df_global_mean[column].astype(str)
        + "\pm"
        + df_global_std[column].astype(str)
        + "$"
    )

print(df_global_mean.to_latex(float_format="%.2f"))

\begin{tabular}{llllllllll}
\toprule
 &  &  & Validity & L2 & Prob. Plaus. & Log Density & IsoForest & LOF & Time \\
model & dataset & method &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{13}{*}{MultilayerPerceptron} & \multirow[t]{2}{*}{BlobsDataset} & GLOBE_CE & $0.99\pm0.01$ & $0.25\pm0.04$ & $0.0\pm0.0$ & $-4.57\pm1.67$ & $-0.06\pm0.03$ & $2.04\pm0.18$ & $0.66\pm0.03$ \\
 &  & RPPCEF_GLOBAL & $1.0\pm0.0$ & $0.48\pm0.01$ & $0.92\pm0.03$ & $2.89\pm0.1$ & $0.03\pm0.0$ & $1.04\pm0.01$ & $7.89\pm0.86$ \\
\cline{2-10}
 & \multirow[t]{2}{*}{DigitsDataset} & GLOBE_CE & $0.0\pm0.0$ & $nan\pmnan$ & $nan\pmnan$ & $nan\pmnan$ & $nan\pmnan$ & $nan\pmnan$ & $0.95\pm0.08$ \\
 &  & RPPCEF_GLOBAL & $1.0\pm0.0$ & $17.08\pm0.54$ & $0.72\pm0.09$ & $-99.42\pm0.61$ & $0.1\pm0.0$ & $1.09\pm0.01$ & $31.48\pm5.28$ \\
\cline{2-10}
 & \multirow[t]{3}{*}{HelocDataset} & AReS & $0.28\pm0.06$ & $0.68\pm0.16$ & $0.18\pm0.14$ & $19.6\pm14.31$ & $0.02\pm0.02$ & $1.23\pm0.09$ & $13.25\pm1.79$ \\
 &  & GLOBE_CE & $1

In [34]:
columns_map = {
    "dataset": "dataset",
    "disc_model": "model",
    "method": "method",
    "K_vectors": "# of Groups",
    "cf_belongs_to_group": "Coverage",
    "validity": "Validity",
    "proximity_continuous_euclidean": "L2",
    "prob_plausibility": "Prob. Plaus.",
    "log_density_cf": "Log Density",
    "isolation_forest_scores_cf": "IsoForest",
    "lof_scores_cf": "LOF",
    "time": "Time",
}
df_group = pd.read_csv("results_GROUP.csv")
print(df_group.columns)
df_group = df_group.rename(columns=columns_map)[columns_map.values()]
df_group.groupby(["model", "dataset", "method"]).mean().round(2)
df_mean = df_group.groupby(["model", "dataset", "method"]).mean().round(2)
df_std = df_group.groupby(["model", "dataset", "method"]).std().round(2)

for column in df_mean.columns:
    df_mean[column] = (
        "$" + df_mean[column].astype(str) + "\pm" + df_std[column].astype(str) + "$"
    )

print(df_mean.to_latex(float_format="%.2f"))

Index(['dataset', 'disc_model', 'method', 'K_vectors', 'validity',
       'prob_plausibility', 'cf_belongs_to_group', 'log_density_cf',
       'proximity_continuous_manhattan', 'proximity_continuous_euclidean',
       'isolation_forest_scores_cf', 'lof_scores_cf', 'time', 'coverage',
       'actionability', 'sparsity', 'proximity_categorical_hamming',
       'proximity_categorical_jaccard', 'proximity_continuous_mad',
       'proximity_l2_jaccard', 'proximity_mad_hamming', 'log_density_test',
       'lof_scores_test', 'isolation_forest_scores_test'],
      dtype='object')
\begin{tabular}{llllllllllll}
\toprule
 &  &  & # of Groups & Coverage & Validity & L2 & Prob. Plaus. & Log Density & IsoForest & LOF & Time \\
model & dataset & method &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{6}{*}{MultilayerPerceptron} & BlobsDataset & RPPCEF & $1.0\pm0.0$ & $1.0\pm0.0$ & $1.0\pm0.0$ & $0.45\pm0.03$ & $0.92\pm0.03$ & $2.88\pm0.1$ & $0.03\pm0.0$ & $1.04\pm0.01$ & $nan\pmnan$ \\
\cline{2-12

In [35]:
columns_map = {
    "dataset": "dataset",
    "disc_model": "model",
    "method": "method",
    "coverage": "Coverage",
    "validity": "Validity",
    "proximity_continuous_euclidean": "L2",
    "prob_plausibility": "Prob. Plaus.",
    "log_density_cf": "Log Density",
    "isolation_forest_scores_cf": "IsoForest",
    "lof_scores_cf": "LOF",
    "time": "Time",
}
df_local = pd.read_csv("results_LOCAL.csv")
print(df_local.columns)
df_local = df_local.rename(columns=columns_map)[columns_map.values()]
df_local.groupby(["model", "dataset", "method"]).mean().round(2)
df_mean = df_local.groupby(["model", "dataset", "method"]).mean().round(2)
df_std = df_local.groupby(["model", "dataset", "method"]).std().round(2)

for column in df_mean.columns:
    df_mean[column] = (
        "$" + df_mean[column].astype(str) + "\pm" + df_std[column].astype(str) + "$"
    )

print(df_mean.to_latex(float_format="%.2f"))

Index(['dataset', 'disc_model', 'method', 'K_vectors', 'validity',
       'prob_plausibility', 'cf_belongs_to_group', 'log_density_cf',
       'proximity_continuous_manhattan', 'proximity_continuous_euclidean',
       'isolation_forest_scores_cf', 'lof_scores_cf', 'time', 'coverage',
       'actionability', 'sparsity', 'proximity_categorical_hamming',
       'proximity_categorical_jaccard', 'proximity_continuous_mad',
       'proximity_l2_jaccard', 'proximity_mad_hamming', 'log_density_test',
       'lof_scores_test', 'isolation_forest_scores_test'],
      dtype='object')
\begin{tabular}{lllllllllll}
\toprule
 &  &  & Coverage & Validity & L2 & Prob. Plaus. & Log Density & IsoForest & LOF & Time \\
model & dataset & method &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{11}{*}{MultilayerPerceptron} & \multirow[t]{2}{*}{BlobsDataset} & RPPCEF_LOCAL & $1.0\pm0.0$ & $1.0\pm0.0$ & $0.39\pm0.01$ & $1.0\pm0.0$ & $2.74\pm0.07$ & $0.03\pm0.0$ & $1.04\pm0.01$ & $6.2\pm0.2$ \\
 &  & wach & $0.9